# 한국어 데이터로 챗봇 만들기
프로젝트 제출 루브릭
| **학습 목표** | **평가 기준** |
|----------------|----------------|
| 한국어 전처리를 통해 학습 데이터셋을 구축하였다. | 공백과 특수문자 처리, 토크나이징, 병렬데이터 구축의 과정이 적절히 진행되었다. |
| 트랜스포머 모델을 구현하여 한국어 챗봇 모델 학습을 정상적으로 진행하였다. | 구현한 트랜스포머 모델이 한국어 병렬 데이터 학습 시 안정적으로 수렴하였다. |
| 한국어 입력문장에 대해 한국어로 답변하는 함수를 구현하였다. | 한국어 입력문장에 맥락에 맞는 한국어로 답변을 리턴하였다. |

# Step 0. Library

In [187]:
#!pip install sentencepiece

In [188]:
import pandas as pd
import sentencepiece as spm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import math
import time
import re
import os

# Step 1. 데이터 수집하기

In [189]:
#!wget https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv
#!mv ChatbotData.csv data/chatbot

데이터 자체가 이런식으로 표현되어 있다.

| **Q** | **A** | **Label** |
|----------------|----------------|----------------|
|12시 땡!, |하루가 또 가네요., |0|

---

데이터의 Label은 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링로 되어있다.

--- 



# Step 2. 데이터 전처리하기

In [190]:
url = 'https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv'
df = pd.read_csv(url)

In [191]:
def  preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    
    return sentence

questions = [preprocess_sentence(q) for q in df['Q']]
answers   = [preprocess_sentence(a) for a in df['A']]

# Step 3. SentencePiece 사용하기

In [192]:
corpus = questions + answers
print('전체 샘플 수 :', len(corpus)/2)
with open('data/chatbot/corpus.txt', 'w', encoding='utf-8') as f:
    for s in corpus:
        f.write(s + '\n')

전체 샘플 수 : 11823.0


In [193]:
spm.SentencePieceTrainer.Train(
    input='data/chatbot/corpus.txt',
    model_prefix='data/chatbot/chatbot_tokenizer',
    vocab_size=8000,
    model_type='bpe',
    character_coverage=1.0,
    pad_id=0, unk_id=1, bos_id=2, eos_id=3
)

sp = spm.SentencePieceProcessor(model_file='data/chatbot/chatbot_tokenizer.model')

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/chatbot/corpus.txt
  input_format: 
  model_prefix: data/chatbot/chatbot_tokenizer
  model_type: BPE
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id: 2
  eos_id: 3
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differen

In [194]:
# 토크나이징 + BOS/EOS
tokenized_questions = [sp.encode_as_ids(q) for q in questions]
tokenized_answers   = [[sp.bos_id()] + sp.encode_as_ids(a) + [sp.eos_id()] for a in answers]

req=10 size=2720 all=35763 active=1862 piece=이니
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=10 size=2740 all=35820 active=1919 piece=▁내요
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=10 size=2760 all=35855 active=1954 piece=▁변명
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=10 size=2780 all=35896 active=1995 piece=▁이뤄
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=10 size=2800 all=35943 active=2042 piece=이라고
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=10 min_freq=6
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=10 size=2820 all=35984 active=1832 piece=▁됐는데
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=10 size=2840 all=35981 active=1829 piece=▁아프다
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=10 size=2860 all=35971 active=1819 piece=▁태어난
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=10 size=2880 all=35980 active=1828 piece=▁생각하면
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=10 size=2900 all=35970 active=1818 piece=▁다르겠지만
bpe_model_trainer.cc(15

# Step 4. 모델 구성하기

## Step 4-1. Model class 선언

In [195]:
class PositionalEncoding(nn.Module):
    def __init__(self, position, d_model):
        super().__init__()
        self.pos_encoding = self._build_pos_encoding(position, d_model)

    def _get_angles(self, pos, i, d_model):
        return pos / (10000 ** ((2 * (i // 2)) / d_model))

    def _build_pos_encoding(self, position, d_model):
        angle_rads = self._get_angles(
            torch.arange(position, dtype=torch.float32).unsqueeze(1),
            torch.arange(d_model, dtype=torch.float32).unsqueeze(0),
            d_model
        )
        sines = torch.sin(angle_rads[:, 0::2])
        cosines = torch.cos(angle_rads[:, 1::2])
        pos_encoding = torch.zeros(position, d_model)
        pos_encoding[:, 0::2] = sines
        pos_encoding[:, 1::2] = cosines
        return pos_encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.pos_encoding[:, :x.size(1), :].to(x.device)

In [196]:
def scaled_dot_product_attention(query, key, value, mask=None):
    matmul_qk = torch.matmul(query, key.transpose(-1, -2))
    depth = key.size(-1)
    logits = matmul_qk / math.sqrt(depth)
    if mask is not None:
        logits = logits.masked_fill(mask == 0, -1e9)
    attention_weights = F.softmax(logits, dim=-1)
    output = torch.matmul(attention_weights, value)
    return output, attention_weights

In [197]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0
        self.depth = d_model // num_heads

        self.query_dense = nn.Linear(d_model, d_model)
        self.key_dense   = nn.Linear(d_model, d_model)
        self.value_dense = nn.Linear(d_model, d_model)
        self.out_dense   = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        query = self.split_heads(self.query_dense(query), batch_size)
        key   = self.split_heads(self.key_dense(key), batch_size)
        value = self.split_heads(self.value_dense(value), batch_size)

        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
        scaled_attention = scaled_attention.permute(0, 2, 1, 3).contiguous()
        concat_attention = scaled_attention.view(batch_size, -1, self.d_model)
        output = self.out_dense(concat_attention)
        return output

In [198]:
def create_padding_mask(x, pad_id=0):
    return (x != pad_id).unsqueeze(1).unsqueeze(2).float()

def create_look_ahead_mask(seq_len):
    mask = torch.tril(torch.ones(seq_len, seq_len))
    return mask

In [199]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_out = self.mha(x, x, x, mask)
        attn_out = self.dropout1(attn_out)
        out1 = self.norm1(x + attn_out)
        ffn_out = self.ffn(out1)
        ffn_out = self.dropout2(ffn_out)
        out2 = self.norm2(out1 + ffn_out)
        return out2

class Encoder(nn.Module):
    def __init__(self, vocab_size, num_layers, ff_dim, d_model, num_heads, max_len=40, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos_encoding = PositionalEncoding(max_len, d_model)
        self.dropout = nn.Dropout(dropout)
        self.enc_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, mask):
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        for layer in self.enc_layers:
            x = layer(x, mask)
        return x

In [200]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.self_mha = MultiHeadAttention(d_model, num_heads)
        self.encdec_mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)
        self.norm3 = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, x, enc_output, look_ahead_mask, padding_mask):
        self_attn = self.self_mha(x, x, x, look_ahead_mask)
        self_attn = self.dropout1(self_attn)
        out1 = self.norm1(x + self_attn)

        encdec_attn = self.encdec_mha(out1, enc_output, enc_output, padding_mask)
        encdec_attn = self.dropout2(encdec_attn)
        out2 = self.norm2(out1 + encdec_attn)

        ffn_out = self.ffn(out2)
        ffn_out = self.dropout3(ffn_out)
        out3 = self.norm3(out2 + ffn_out)
        return out3

class Decoder(nn.Module):
    def __init__(self, vocab_size, num_layers, ff_dim, d_model, num_heads, max_len=200, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos_encoding = PositionalEncoding(max_len, d_model)
        self.dropout = nn.Dropout(dropout)
        self.dec_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, enc_output, look_ahead_mask, padding_mask):
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        for layer in self.dec_layers:
            x = layer(x, enc_output, look_ahead_mask, padding_mask)
        return x

In [201]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, num_layers=6, units=2048, d_model=512, num_heads=8, dropout=0.1, max_len=200):
        super().__init__()
        self.encoder = Encoder(vocab_size, num_layers, units, d_model, num_heads, max_len, dropout)
        self.decoder = Decoder(vocab_size, num_layers, units, d_model, num_heads, max_len, dropout)
        self.final_linear = nn.Linear(d_model, vocab_size)

    def forward(self, inputs, dec_inputs):
        enc_padding_mask = create_padding_mask(inputs, sp.pad_id())
        look_ahead_mask = create_look_ahead_mask(dec_inputs.size(1)).to(inputs.device)
        look_ahead_mask = look_ahead_mask.unsqueeze(0).unsqueeze(1)
        dec_padding_mask = create_padding_mask(inputs, sp.pad_id())

        enc_output = self.encoder(inputs, enc_padding_mask)
        dec_output = self.decoder(dec_inputs, enc_output, look_ahead_mask, dec_padding_mask)
        logits = self.final_linear(dec_output)
        return logits

## Step 4-2. Dataset & DataLoader

In [202]:
class ChatbotDataset(Dataset):
    def __init__(self, qs, ans):
        self.qs, self.ans = qs, ans
    def __len__(self): return len(self.qs)
    def __getitem__(self, i):
        return torch.tensor(self.qs[i]), torch.tensor(self.ans[i])

def collate(batch):
    src, tgt = zip(*batch)
    src = pad_sequence(src, batch_first=True, padding_value=sp.pad_id())
    tgt = pad_sequence(tgt, batch_first=True, padding_value=sp.pad_id())
    return src, tgt


# 전체 데이터셋
full_dataset = ChatbotDataset(tokenized_questions, tokenized_answers)

# 80% train, 10% valid, 10% test
train_size = int(0.8 * len(full_dataset))
valid_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split(
    full_dataset, [train_size, valid_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, collate_fn=collate)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, collate_fn=collate)

print(f"Train: {len(train_dataset)} | Valid: {len(valid_dataset)} | Test: {len(test_dataset)}")

Train: 9458 | Valid: 1182 | Test: 1183


## Step 4-3. Model Train

In [203]:
def get_lr_lambda(d_model, warmup_steps=4000):
    """
    Transformer 논문의 학습률 스케줄링
    lr = d_model^(-0.5) * min(step^(-0.5), step * warmup_steps^(-1.5))
    """
    d_model = float(d_model)
    def lr_lambda(step):
        step = step + 1  # 0-based -> 1-based
        return (d_model ** -0.5) * min(step ** -0.5, step * (warmup_steps ** -1.5))
    return lr_lambda

In [204]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
VOCAB_SIZE = sp.get_piece_size()

model = Transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=3,
    units=512,
    d_model=256,
    num_heads=8,
    dropout=0.1,
    max_len=100
).to(device)

optimizer = optim.Adam(model.parameters(), lr=1.0, betas=(0.9, 0.98), eps=1e-9)  # lr은 scheduler가 조절
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=get_lr_lambda(d_model=256, warmup_steps=4000))

criterion = nn.CrossEntropyLoss(ignore_index=sp.pad_id())

In [205]:
# 모델 저장 경로
os.makedirs("data/chatbot/checkpoints", exist_ok=True)
best_val_loss = float('inf')

In [206]:
@torch.no_grad()
def evaluate(model, loader, crit):
    model.eval()
    total_loss = 0.0
    for src, tgt in loader:
        src, tgt = src.to(device), tgt.to(device)
        tgt_input = tgt[:, :-1]
        tgt_target = tgt[:, 1:].contiguous().view(-1)
        logits = model(src, tgt_input)
        loss = crit(logits.view(-1, VOCAB_SIZE), tgt_target)
        total_loss += loss.item()
    return total_loss / len(loader)

In [207]:
def train_one_epoch(model, loader, opt, scheduler, crit):
    model.train()
    total_loss = 0.0
    for step, (src, tgt) in enumerate(loader):
        src, tgt = src.to(device), tgt.to(device)
        tgt_input = tgt[:, :-1]
        tgt_target = tgt[:, 1:].contiguous().view(-1)

        opt.zero_grad()
        logits = model(src, tgt_input)
        loss = crit(logits.view(-1, VOCAB_SIZE), tgt_target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        scheduler.step()

        total_loss += loss.item()

        if (step + 1) % 100 == 0:
            print(f"  Step {step+1} | Loss: {loss.item():.4f} | LR: {scheduler.get_last_lr()[0]:.6f}")

    return total_loss / len(loader)

In [208]:
N_EPOCHS = 20
for epoch in range(1, N_EPOCHS + 1):
    start = time.time()
    train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, criterion)
    val_loss = evaluate(model, valid_loader, criterion)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "data/chatbot/checkpoints/best_transformer.pth")
        print(f"  [SAVE] Best model saved | Val Loss: {val_loss:.4f}")

    print(f'Epoch {epoch:02d} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | '
          f'PPL: {math.exp(val_loss):6.2f} | {time.time()-start:.1f}s\n')

  Step 100 | Loss: 7.3118 | LR: 0.000025
  [SAVE] Best model saved | Val Loss: 6.6955
Epoch 01 | Train: 7.9545 | Val: 6.6955 | PPL: 808.73 | 4.6s

  Step 100 | Loss: 6.3815 | LR: 0.000062
  [SAVE] Best model saved | Val Loss: 6.0060
Epoch 02 | Train: 6.2779 | Val: 6.0060 | PPL: 405.86 | 4.6s

  Step 100 | Loss: 5.7508 | LR: 0.000098
  [SAVE] Best model saved | Val Loss: 5.7327
Epoch 03 | Train: 5.7864 | Val: 5.7327 | PPL: 308.81 | 4.5s

  Step 100 | Loss: 5.1924 | LR: 0.000135
  [SAVE] Best model saved | Val Loss: 5.5516
Epoch 04 | Train: 5.5198 | Val: 5.5516 | PPL: 257.64 | 4.5s

  Step 100 | Loss: 5.1703 | LR: 0.000171
  [SAVE] Best model saved | Val Loss: 5.3567
Epoch 05 | Train: 5.2556 | Val: 5.3567 | PPL: 212.03 | 4.5s

  Step 100 | Loss: 5.0901 | LR: 0.000208
  [SAVE] Best model saved | Val Loss: 5.1893
Epoch 06 | Train: 4.9614 | Val: 5.1893 | PPL: 179.35 | 4.4s

  Step 100 | Loss: 4.5547 | LR: 0.000244
  [SAVE] Best model saved | Val Loss: 5.0302
Epoch 07 | Train: 4.6726 | Val: 

# Step 5. 모델 평가하기

In [209]:
model.load_state_dict(torch.load("data/chatbot/checkpoints/best_transformer.pth"))
print("Best model loaded.\n")

test_loss = evaluate(model, test_loader, criterion)
print(f"=== FINAL TEST LOSS: {test_loss:.4f} | PPL: {math.exp(test_loss):6.2f} ===\n")

Best model loaded.

=== FINAL TEST LOSS: 4.4010 | PPL:  81.53 ===



In [210]:
@torch.no_grad()
def respond(model, sp, sentence, max_len=50):
    model.eval()
    src = torch.tensor([sp.encode_as_ids(sentence)], device=device)
    enc_padding_mask = create_padding_mask(src, sp.pad_id())
    enc_output = model.encoder(src, enc_padding_mask)

    dec_input = torch.tensor([[sp.bos_id()]], device=device)
    output_ids = []

    for _ in range(max_len):
        look_ahead_mask = create_look_ahead_mask(dec_input.size(1)).to(device)
        look_ahead_mask = look_ahead_mask.unsqueeze(0).unsqueeze(1)
        dec_padding_mask = create_padding_mask(src, sp.pad_id())

        dec_output = model.decoder(dec_input, enc_output, look_ahead_mask, dec_padding_mask)
        logits = model.final_linear(dec_output[:, -1, :])
        next_id = logits.argmax(dim=-1).item()

        output_ids.append(next_id)
        if next_id == sp.eos_id():
            break

        dec_input = torch.cat([dec_input, torch.tensor([[next_id]], device=device)], dim=1)

    return sp.decode_ids(output_ids)

In [211]:
test_indices = test_dataset.indices  # 원본 df에서의 인덱스 리스트
test_questions = [df.Q[idx] for idx in test_indices]
test_answers   = [df.A[idx] for idx in test_indices]

for i in range(len(test_questions)):
    q = test_questions[i]
    real_a = test_answers[i]
    pred_a = respond(model, sp, q)

    print(f"\nQ: {q}")
    print(f"실제 A: {real_a}")
    print(f"모델 A: {pred_a}")


Q: 둘 다 바보였다
실제 A: 바보의 사랑이네요.
모델 A: 좋은 생각입니다 .

Q: 일만 하고 살았는데 이 모양
실제 A: 자신의 인생을 사세요.
모델 A: 잠깐 핸드폰 게임할 수도 있어요 .

Q: 쉬는날인데 왜 만나자고 안하지
실제 A: 직접 만나자고 연락해보세요.
모델 A: 잠시 쉬어도 괜찮아요 .

Q: 비정규직이라 차별받아
실제 A: 차별이 가장 안 좋은 건데 슬프네요.
모델 A: 이제 취업 하셔야죠 .

Q: 오늘도 공기가 뿌얘
실제 A: 마스크 착용 하시고 외출하세요.
모델 A: 이제 좀 괜찮아졌길 바랍니다 .

Q: 만난지 얼마 안됐는데 고백받았어
실제 A: 정말 좋아했나봐요.
모델 A: 한 번 떠난 마음은 알기 어렵죠 .

Q: 파마할까
실제 A: 한번 해보세요.
모델 A: 한번 하세요 .

Q: 젊어지고 싶어
실제 A: 마음이라도 어리게 사세요.
모델 A: 맛있게 드세요 .

Q: 자기 소개 또 어떻게 해
실제 A: 강렬하고 자신감있게 하면 되지 않을까싶어요.
모델 A: 후회해도 늦었어요 .

Q: 그게 잘 안되네.
실제 A: 마음이 쉽지 않죠.
모델 A: 잘할 수 있을 거예요 .

Q: 그녀는 나를 어떻게 생각할까?
실제 A: 아직은 잘 모르겠어요.
모델 A: 정말 안녕 .

Q: 답정녀 좀 싫어
실제 A: 못 이기니 피할 수 있으면 피하세요.
모델 A: 나쁜 생각 하지 마세요 .

Q: 지금이 겨울이라 참 다행이이네
실제 A: 제가 곁에 있을게요.
모델 A: 너무 걱정이네요 .

Q: 씁쓸하네.
실제 A: 제가 있잖아요.
모델 A: 자신을 이겨야해요 .

Q: 짝남의 여사친들 보기 질투나.
실제 A: 질투난다고 살짝 말해봐요.
모델 A: 제가 들어드릴게요 .

Q: 그녀의 환심을 살 수 있는 방법이 뭐가 있을까?
실제 A: 그녀가 좋아하는게 뭔지 찾아봐요.
모델 A: 잘 살 수 있을 거예요 .

Q: 사랑해서 너무 행복해
실제 A: 어쩐지 행복해 보이더라구요.
모델 A: 사랑하는 이유를 써를 가지고 손해 .

Q: 미친것같아
실제 